In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Water Quality Dataset**

This dataset contains water quality for 3276 different water bodies.

The Contents Are : 

**1. pH Value**
**2. Hardness**
**3. Solids**
**4. Chloramines**
**5. Sulfate**
**6. Conductivity**
**7. Organic Carbon**
**8. Thrihalomethanes**
**9. Turbidity**
**10. Potability**

**Pre-Processing**

In [ ]:
data_water = pd.read_csv('../input/water-potability/water_potability.csv')
data_water.head()

In [ ]:
data_water.info()

In [ ]:
data_water.isnull().sum()

In [ ]:
data_water[data_water.ph.isnull()][data_water.Sulfate.isnull()][data_water.Trihalomethanes.isnull()].head()

In [ ]:
data_water[data_water['Potability'] == 0][['ph', 'Sulfate', 'Trihalomethanes']].describe()

In [ ]:
data_water[data_water['Potability'] == 1][['ph', 'Sulfate', 'Trihalomethanes']].describe()

The difference between the value of column (ph, Sufate, Trihalomethaneds) are not too diferent, so I will fill the missing values with the mean value

In [ ]:
data_water['ph'] = data_water['ph'].fillna(data_water['ph'].mean())
data_water['Sulfate'] = data_water['Sulfate'].fillna(data_water['Sulfate'].mean())
data_water['Trihalomethanes'] = data_water['Trihalomethanes'].fillna(data_water['Trihalomethanes'].mean())
data_water.isnull().sum()

Okay, the missing values are handled and right now I am deciding on what to do with the data.
The plans are one of this below :
* Creating MLP using Neural Network to predict Potability of water
* Analyzing in-depth of factors in water potability

**MLP using Neural Network / Regression to Predict Potability of Water**

Splitting Train and Test Data

In [ ]:
X_train,X_test=train_test_split(data_water)
print(len(X_train))
len(X_test)

Shuffling X & Creating y train and y test

In [ ]:
X_train_shuffled = shuffle(X_train)
X_test_shuffled = shuffle(X_test)
y_train_shuffled = X_train_shuffled['Potability']
y_test_shuffled = X_test_shuffled['Potability']

In [ ]:
X_train_shuffled = X_train_shuffled.drop(['Potability'], axis = 1)
X_test_shuffled = X_test_shuffled.drop(['Potability'], axis = 1)
print(X_train_shuffled)
print(X_test_shuffled)

Data Normalization

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_shuffled)
X_test_scaled = scaler.fit_transform(X_test_shuffled)

Regression Model

In [ ]:
modelling_XGB = XGBClassifier(use_label_encoder = False)
modelling_XGB.fit(X_train_scaled, y_train_shuffled)

In [ ]:
predict = modelling_XGB.predict(X_test_scaled)
result = f1_score(y_test_shuffled, predict)
print(result)

It seems that using XGB is not enough to predict because the result is way below average (<50%)

Let's try other type of MLP Train Methods

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train_shuffled)
y_pred = logreg.predict(X_test_scaled)
acc = accuracy_score(y_test_shuffled, y_pred)
print(acc)

By using logistic regression, we can already get even better result with 0.61 accuracy

Let's try another method

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train_shuffled)
y_pred = rfc.predict(X_test_scaled)
acc = accuracy_score(y_test_shuffled, y_pred)
print(acc)

In [ ]:
svc = SVC()
svc.fit(X_train_scaled, y_train_shuffled)
y_pred = svc.predict(X_test_scaled)
acc = accuracy_score(y_test_shuffled, y_pred)
print(acc)

In [ ]:
kneighbors = KNeighborsClassifier()
kneighbors.fit(X_train_scaled, y_train_shuffled)
y_pred = kneighbors.predict(X_test_scaled)
acc = accuracy_score(y_test_shuffled, y_pred)
print(acc)